<a href="https://colab.research.google.com/github/Angadkr/Image_to_catalogue/blob/main/Image_to_catalogue.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# ✅ STEP 1: Setup + Mount Google Drive

from google.colab import drive
import os

# Mount Google Drive
drive.mount('/content/drive')

# Create project directories in Drive
project_root = "/content/drive/MyDrive/cloth_catalog_project"
input_folder = os.path.join(project_root, "input_images")
output_folder = os.path.join(project_root, "output_catalogs")

os.makedirs(input_folder, exist_ok=True)
os.makedirs(output_folder, exist_ok=True)

print(f"Upload your raw images to: {input_folder}")
print(f"Processed catalog images will be saved to: {output_folder}")


Mounted at /content/drive
Upload your raw images to: /content/drive/MyDrive/cloth_catalog_project/input_images
Processed catalog images will be saved to: /content/drive/MyDrive/cloth_catalog_project/output_catalogs


In [ ]:
import os
import subprocess

# Use local working directory
project_dir = os.path.join(os.getcwd(), "u2net_catalog_project")
os.makedirs(project_dir, exist_ok=True)

# Clone U²-Net repository
repo_path = os.path.join(project_dir, "U-2-Net")
if not os.path.exists(repo_path):
    subprocess.run(["git", "clone", "https://github.com/xuebinqin/U-2-Net.git", repo_path])

# Download the model
model_dir = os.path.join(repo_path, "saved_models", "u2net")
os.makedirs(model_dir, exist_ok=True)
model_path = os.path.join(model_dir, "u2net.pth")

if not os.path.exists(model_path):
    subprocess.run([
        "wget", "-O", model_path,
        "https://github.com/xuebinqin/U-2-Net/releases/download/v1.0/u2net.pth"
    ])

print("✅ U²-Net setup complete.")


✅ U²-Net setup complete.


In [ ]:
import torch
from torch.autograd import Variable
from torchvision import transforms
from PIL import Image
import numpy as np
import os
import cv2
import sys

# ✅ Add model path to Python path
sys.path.append("u2net_catalog_project/U-2-Net")

# ✅ Import model
from model import U2NET  # or from model import U2NETP for small version

# ✅ Load model
model_path = "u2net_catalog_project/U-2-Net/saved_models/u2net/u2net.pth"
net = U2NET(3, 1)
net.load_state_dict(torch.load(model_path, map_location='cpu'))
net.eval()

print("✅ U²-Net model loaded.")


✅ U²-Net model loaded.


In [ ]:
# Preprocessing
transform = transforms.Compose([
    transforms.Resize((320, 320)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225])
])

def norm_pred(d):
    ma = torch.max(d)
    mi = torch.min(d)
    dn = (d - mi) / (ma - mi)
    return dn

def save_output(image_name, pred, output_dir):
    pred = pred.squeeze().cpu().data.numpy()
    mask = (pred * 255).astype(np.uint8)
    mask = cv2.resize(mask, (orig_w, orig_h))
    mask = cv2.merge([mask, mask, mask])

    # Open original image
    orig = cv2.imread(image_name)
    orig = cv2.resize(orig, (orig_w, orig_h))

    # Composite over white
    white_bg = np.ones_like(orig) * 255
    alpha = mask[:, :, 0] / 255.0
    comp = alpha[..., None] * orig + (1 - alpha[..., None]) * white_bg

    # Save
    cv2.imwrite(os.path.join(output_dir, os.path.basename(image_name)), comp)
    print(f"✅ Saved: {os.path.basename(image_name)}")


In [ ]:
input_path = "u2net_catalog_project/input_images/test.jpg"
output_dir = "u2net_catalog_project/output_catalogs"
os.makedirs(output_dir, exist_ok=True)

# Original size
img = Image.open(input_path).convert('RGB')
orig_w, orig_h = img.size

# Preprocess
image_tensor = transform(img).unsqueeze(0)
image_tensor = Variable(image_tensor)

# Run model
with torch.no_grad():
    d1, _, _, _, _, _, _ = net(image_tensor)

# Normalize + Save
pred = norm_pred(d1[:, 0, :, :])
save_output(input_path, pred, output_dir)


✅ Saved: test.jpg
